# Unit testing tutorial

This unit test tutorial takes place inside a Jupyter notebook.
All tests are real.
The only thing we have to do is to write a little "test runner".
Usually, you will use something like `pytest` to run your tests.
However, here we run our tests inside a notebook.

Every single test is supposed to fail!
The only interesting thing about tests is how they fail.

In [ ]:
import unittest

def test(klass):
    loader = unittest.TestLoader()
    suite=loader.loadTestsFromTestCase(klass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

Tests can *fail* or *error*. These are different things.
A test error happens when an exception is raised.

In [ ]:
class TestSimpleFailure(unittest.TestCase):
    def test_fail(self):
        1/0

test(TestSimpleFailure)

A test *failure* happens when a test assertion is violated.
This means that to fail tests,
we should write assertions.
The `unittest` library comes with its own assertions,
but those are a bit rigid and unpleasant to use.

`pytest` offers a mini-language that looks like Python:
it will look at your `assert` statements,
and generate assertions from that.
This can sometimes be obscure:
when it works, it works great,
but when it violates expectations,
it is somewhat obscure why.

In contract, `PyHamcrest` is a flexible assertion library.
It does not try to be a unit test framework
*or*
a unit test runner.
Its only goal is to be the best assertion library.

In [ ]:
from hamcrest import assert_that, equal_to

In [ ]:
class TestEquality(unittest.TestCase):
    def test_paradox(self):
        assert_that(1, equal_to(0))

test(TestEquality)

Often it turns out we need to construct some object the same way for multiple tests.
Such
*fixtures*
can be constructed using `setUp`.
In this example,
we build the object `1`.

In [ ]:
class TestMultipleEquality(unittest.TestCase):
    def setUp(self):
        self.value = 1
    def test_paradox(self):
        assert_that(self.value, equal_to(0))
    def test_anoter_paradox(self):
        assert_that(self.value, equal_to(2))

test(TestMultipleEquality)

Some fixtures need
*cleanup*.
For example,
files should be closed.

In [ ]:
class TestRead(unittest.TestCase):
    def test_read(self):
        self.addCleanup(lambda: fpin.close())
        fpin = open("/dev/zero")
        assert_that(fpin.read(4), equal_to(""))

test(TestRead)

If we need a cleanup-needing fixture for multiple tests,
we can move the creation/cleanup logic to `setUp`.
This is nothing more than what we already know:
`addCleanup` runs at the end of a test,
and `setUp` runs before each test.

In [ ]:
class TestDoubleRead(unittest.TestCase):
    def setUp(self):
        self.addCleanup(lambda: self.fpin.close())
        self.fpin = open("/dev/zero")
        
    def test_short_read(self):
        assert_that(self.fpin.read(4), equal_to(""))
        
    def test_long_read(self):
        assert_that(self.fpin.read(8), equal_to(""))        

test(TestDoubleRead)

If the tests are so similar as to only differ by a parameter,
we can use
*subtests*.
Subtests share all the logic,
but they do not share failures.
Note that a subtest failing does not halt execution of the test function.

In [ ]:
class TestMultiRead(unittest.TestCase):
    def test_all_reads(self):
        self.addCleanup(lambda: fpin.close())
        fpin = open("/dev/zero")
        for length in (1, 4, 8):
            with self.subTest(length=length):
                assert_that(fpin.read(length), equal_to(""))

test(TestMultiRead)

Note that in this case,
the same open file was used in all tests.
If we want separate fixtures for subtests,
we need to create them separately.

In [ ]:
class TestMultiSeparateRead(unittest.TestCase):
    def test_all_reads(self):
        for length in (1, 4, 8):
            with self.subTest(length=length):
                with open("/dev/zero") as fpin:
                    assert_that(fpin.read(length), equal_to(""))

test(TestMultiSeparateRead)

PyHamcrest has a lot of useful matchers. Here are some of them

In [ ]:
class TestInterestingMatchers(unittest.TestCase):
    pass

test(TestInterestingMatchers)

One powrful way to build projects is by using the `mock` library.

In [ ]:
class TestMockingConstructs(unittest.TestCase)